In [ ]:
!pip install yfinance --upgrade --no-cache-dir -q
!pip install pandas_datareader -q

     |████████████████████████████████| 6.4 MB 5.9 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import yfinance
import requests
import lxml
import pandas_datareader
from pandas_datareader import data as pdr
import yfinance as yf
from torch.nn.functional import softmax
from tqdm import tqdm

yf.pdr_override() # <== that's all it takes :-)

def financial_dataset(stock, num_of_labels=2, cutoff=0.25,
                      start_date="2010-01-01", end_date="2021-01-01") :
    ''' Downloads financial data for a stock and process it in the desired format
        Parameters :
          stock(str) : The desired stock's code
          cutoff(float) : A float indicating a percentage under which no price change is considered                                 increase or decrease eg. 0.25 = 0.25% price change from close-to-close
          num_of_labels(2 or 3) : Number of labels to use. 2 = [Increase,Decrease]. 
                                  3=[Increase, Decrease, Sideways]
          start_date(str) : "year-month-day" The day data collection will start .
          end_date(str) : "year-month-day" The day data collection will stop .    '''
    # parameter value check
    if (num_of_labels < 2 or num_of_labels > 3): 
        return print('Number of labels can be either 2 or 3')
                                                            
    fin_data = pdr.get_data_yahoo(stock, start=start_date, end=end_date)
    
    print(f"{stock} financial dataframe dimensions ", fin_data.shape)
    
    # initialize price_change column 
    fin_data['Price_change'] = 1
    fin_data['date'] = 0
    dates = fin_data.index
    yesterday = str(dates[0].date())

    # How much should the price change in abs value to be considered increase/decrease.  
    for date in dates[1:] :
        today = str(date.date())

        yesterday_pr = fin_data.loc[yesterday, 'Close']
        today_pr = fin_data.loc[today, 'Close']
        diff = 100 * (today_pr - yesterday_pr)/yesterday_pr

        if (num_of_labels == 3) :
            if (diff > cutoff) :
                # price increase
                price_change = +1
            elif (diff < -cutoff) :
                # price decrease
                price_change = -1
            else:
                # almost steady price
                price_change = 0 
        elif (num_of_labels == 2 ): 
            if (diff > 0 ) : 
                # price increase
                price_change = +1
            elif (diff <= 0 ) :
                price_change = -1 
                                                                                                       
        yesterday = today
        fin_data.loc[today,'Price_change'] = price_change
        fin_data.loc[today,'date'] = today

    incr = fin_data[fin_data['Price_change'] == 1 ].shape[0]
    decr = fin_data[fin_data['Price_change'] == -1 ].shape[0]
    stable = fin_data[fin_data['Price_change'] == 0 ].shape[0]
    print(f'Positive changes : {incr}')
    print(f'Negative changes : {decr}')
    print(f'No changes : {stable}')

    fin_data = fin_data.drop(columns = ['Low', 'High', 'Adj Close'], axis=1)
        
    return fin_data

def read_news(stock):
    def read_rph(stock) :
        ''' Reads news relevant to 'stock' from the "raw_partner_headlines.csv" csv file. 
            Returns a dataframe in the format :[ Headline | date | stock  ] '''

        csv_path = '/content/drive/MyDrive/Dataset/Datasets/Fin_datasets/raw_partner_headlines.csv'
        arp = pd.read_csv(csv_path)
        arp = arp.drop(columns=['Unnamed: 0', 'url', 'publisher'], axis=1)
        # Format the date column to match financial dataset
        arp['date'] = arp['date'].apply(lambda x: x.split(' ')[0] )
        news = arp[arp['stock'] == stock]
        print(f"The bot found {news.shape[0]} headlines from raw_partner_headlines.csv, regarding {stock} stock")
        return news

    def read_arp(stock) :
        ''' Reads news relevant to 'stock' from the "analyst_rating_processed.csv" csv file. 
        Returns a dataframe in the format :[ Headline | date | stock  ] '''
        csv_path = '/content/drive/MyDrive/Dataset/Datasets/Fin_datasets/analyst_ratings_processed.csv'
        arp = pd.read_csv(csv_path)
        arp = arp.drop(columns=['Unnamed: 0'], axis=1)
        # pick the stock headlines
        arp = arp[arp['stock'] == stock]
        # Format the date column to match financial dataset (only keep date, not time)
        arp['date'] = arp['date'].apply(lambda x: str(x).split(' ')[0] )
        # Rename column title to headline to match other csv
        arp.rename({'title': 'headline'}, axis=1, inplace=True)
        news = arp
        print(f"The bot found {news.shape[0]} headlines from analyst_ratings_processed.csv, regarding {stock} stock")
        return news
    
    arp = read_arp(stock)
    rph = read_rph(stock)
    news = pd.concat([rph, arp], ignore_index=True)
    print(f"The bot found {news.shape[0]} headlines in total, regarding {stock} stock")
    return news
    

def merge_fin_news(df_fin, df_news, how='inner') :
    ''' Merges the financial data dataframe with the news dataframe and rearranges the column order
        how(str) : Merging technique : 'inner', 'outer' etc.. (check pd.merge documentation)      '''
    # merge on date column and only for their intersection
    merged_df = df_fin.merge(df_news, on='date', how=how)
    # rearrange column order
    merged_df = merged_df[['date', 'stock', 'Open', 'Close', 'Volume',  'headline', 'Price_change']]
    return merged_df

def sentim_analyzer(df, tokenizer, model):
    ''' Given a df that contains a column 'headline' with article healine texts, it runs inference on the healine with the 'model' (FinBert) 
       and inserts output sentiment features into the dataframe in the respective columns (Positive_sentim, Negative_sentim, Neutral_sentim)
       
        Parameters :
          df : A dataframe that contains headlines in a column called 'headline' . 
          tokenizer(AutoTokenizer object) : A pre-processing tokenizer object from Hugging Face lib. 
          model (AutoModelForSequenceClassification object) : A hugging face transformer model.     
          
          returns df : The initial dataframe with the 3 sentiment features as columns for each headline'''
    
    for i in tqdm(df.index) :
        try:
            headline = df.loc[i, 'headline']
        except:
            return print(' \'headline\' column might be missing from dataframe')
        # Pre-process input phrase
        input = tokenizer(headline, padding = True, truncation = True, return_tensors='pt')
        # Estimate output
        output = model(**input)
        # Pass model output logits through a softmax layer.
        predictions = softmax(output.logits, dim=-1)
        df.loc[i, 'Positive'] = predictions[0][0].tolist()
        df.loc[i, 'Negative'] = predictions[0][1].tolist()
        df.loc[i, 'Neutral']  = predictions[0][2].tolist()
    # rearrange column order
    try:
        df = df[['date', 'stock', 'Open', 'Close', 'Volume',  'headline', 'Positive', 'Negative', 'Neutral','Price_change']]
    except:
        pass
    return df

def merge_dates(df):
    '''
    Given a df that contains columns [date, stock, Open, Close, Volume, headline, Positive, Negative, Neutral, Price_change],
    take the average of Positive, Negative, Neutral sentiment scores for each date and return a df that contains each
    date exactly one time. The return df has no column 'headline' since the scores now refer to an average of multiple
    news headlines.
        Parameters :
          df : A dataframe with columns [date, stock, Open, Close, Volume, headline, Positive, Negative, Neutral, Price_change]
          returns df : aggragated sentiment scores by date with columns [date, stock, Open, Close, Volume, headline, Positive, Negative, Neutral, Price_change]
    '''

    # read the full enriched dataset in your main code like below and then pass it to the function
    # df = pd.read_csv('Financial_News/train_apple.csv', index_col=0, parse_dates=['date'])

    # take the average for Positive, Negative and Neutral columns by date. Drop headline column and all other columns per date are identical.
    dates_in_df = df['date'].unique()
    new_df = df.copy(deep=True).head(0)  # just take the df structure with no data inside
    new_df = new_df.drop(columns=['headline'])  # drop headline column

    for date in dates_in_df:
        sub_df = df[df['date'] == date]  # filter specific dates
        avg_positive = sub_df['Positive'].mean()
        avg_negative = sub_df['Negative'].mean()
        avg_neutral = sub_df['Neutral'].mean()
        sub_df = sub_df.drop(columns=['headline'])  # drop headline column

        stock = sub_df.iloc[0]['stock']
        open = sub_df.iloc[0]['Open']
        close = sub_df.iloc[0]['Close']
        volume = sub_df.iloc[0]['Volume']
        price_change = sub_df.iloc[0]['Price_change']

        sub_df = sub_df.head(0)  # empty sub_df to populate with just 1 row for each date
        # print(sub_df)
        sub_df.loc[0] = [date, stock, open, close, volume, avg_positive, avg_negative, avg_neutral,
                         price_change]  # populate the row
        # add sub_df's row to the new dataframe
        new_df = pd.concat([new_df, sub_df], axis=0, ignore_index=True)
    print(f" Dataframe now contains sentiment score for {new_df.shape[0]} different dates.")
    return(new_df)

In [ ]:
!pip install transformers -q

     |████████████████████████████████| 3.8 MB 3.9 MB/s 
     |████████████████████████████████| 6.5 MB 29.8 MB/s 
     |████████████████████████████████| 67 kB 3.3 MB/s 
     |████████████████████████████████| 895 kB 38.9 MB/s 
     |████████████████████████████████| 596 kB 45.4 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
#['AAPL', 'MSFT','AMZN','GOOG', 'TSLA', 'NVDA',
#stocks =         ['BRK', 'FB', 'JPM',
        #  'UNH', 'JNJ', 'PG', 'V', 'HD','BAC', 'XOM', 'MA', 'DIS', 'PFE', 'CVX',
         # 'ABBV', 'KO', 'CSCO', 'AVGO', 'VZ', 'PEP', 'WMT', 
stocks =['ADBE', 'INTC', 'AMD', 'QCOM', 'MCD',
          'NFLX', 'NKE', 'UPS', 'AMAT', 'AXP', 'GS'] 

In [ ]:
#big_df = pd.DataFrame()
# Read finances and news for all stocks in list and create a big dataframe.
for stock in stocks :
    news = read_news(stock)
    finances = financial_dataset(stock, num_of_labels=2)
    merged_df = merge_fin_news(finances, news)
    print(f"Performing sentiment analysis for stock {stock}")
    sentimentized_df = sentim_analyzer(merged_df, tokenizer, model)
    # average sentiment for each unique date.
    sentimentized_df = merge_dates(sentimentized_df)
    big_df = pd.concat([big_df, sentimentized_df], axis=0, ignore_index=True)
    big_df.to_csv('/content/drive/MyDrive/Dataset/Datasets/sp500.csv')

The bot found 1992 headlines from analyst_ratings_processed.csv, regarding ADBE stock
The bot found 127 headlines from raw_partner_headlines.csv, regarding ADBE stock
The bot found 2119 headlines in total, regarding ADBE stock
[*********************100%***********************]  1 of 1 completed
ADBE financial dataframe dimensions  (2769, 6)
Positive changes : 1483
Negative changes : 1286
No changes : 0
Performing sentiment analysis for stock ADBE


100%|██████████| 2050/2050 [03:49<00:00,  8.93it/s]


 Dataframe now contains sentiment score for 744 different dates.
The bot found 10 headlines from analyst_ratings_processed.csv, regarding INTC stock
The bot found 2465 headlines from raw_partner_headlines.csv, regarding INTC stock
The bot found 2475 headlines in total, regarding INTC stock
[*********************100%***********************]  1 of 1 completed
INTC financial dataframe dimensions  (2769, 6)
Positive changes : 1431
Negative changes : 1338
No changes : 0
Performing sentiment analysis for stock INTC


100%|██████████| 2286/2286 [04:10<00:00,  9.13it/s]


 Dataframe now contains sentiment score for 394 different dates.
The bot found 250 headlines from analyst_ratings_processed.csv, regarding AMD stock
The bot found 0 headlines from raw_partner_headlines.csv, regarding AMD stock
The bot found 250 headlines in total, regarding AMD stock
[*********************100%***********************]  1 of 1 completed
AMD financial dataframe dimensions  (2769, 6)
Positive changes : 1375
Negative changes : 1394
No changes : 0
Performing sentiment analysis for stock AMD


...

 Dataframe now contains sentiment score for 367 different dates.
The bot found 1852 headlines from analyst_ratings_processed.csv, regarding AXP stock
The bot found 1926 headlines from raw_partner_headlines.csv, regarding AXP stock
The bot found 3778 headlines in total, regarding AXP stock
[*********************100%***********************]  1 of 1 completed
AXP financial dataframe dimensions  (2769, 6)
Positive changes : 1434
Negative changes : 1335
No changes : 0
Performing sentiment analysis for stock AXP


100%|██████████| 3536/3536 [06:16<00:00,  9.40it/s]


 Dataframe now contains sentiment score for 1372 different dates.
The bot found 0 headlines from analyst_ratings_processed.csv, regarding GS stock
The bot found 0 headlines from raw_partner_headlines.csv, regarding GS stock
The bot found 0 headlines in total, regarding GS stock
[*********************100%***********************]  1 of 1 completed
GS financial dataframe dimensions  (2769, 6)
Positive changes : 1423
Negative changes : 1346
No changes : 0
Performing sentiment analysis for stock GS


0it [00:00, ?it/s]

 Dataframe now contains sentiment score for 0 different dates.


In [ ]:
big_df

,date,stock,Open,Close,Volume,Positive,Negative,Neutral,Price_change
0,2020-03-09,AAPL,65.937500,66.542503,286744800.0,0.046127,0.411464,0.542409,-1
1,2020-03-10,AAPL,69.285004,71.334999,285290000.0,0.070845,0.449025,0.480130,1
2,2020-03-11,AAPL,69.347504,68.857498,255598800.0,0.190995,0.453761,0.355244,-1
3,2020-03-12,AAPL,63.985001,62.057499,418474000.0,0.204221,0.447518,0.348261,-1
4,2020-03-13,AAPL,66.222504,69.492500,370732000.0,0.315863,0.218127,0.466010,1
...,...,...,...,...,...,...,...,...,...
20190,2020-06-04,AXP,103.470001,106.540001,5841600.0,0.876779,0.016645,0.106576,1
20191,2020-06-05,AXP,114.110001,109.730003,10097600.0,0.929944,0.026850,0.043206,1
20192,2020-06-08,AXP,111.910004,113.669998,6795200.0,0.897881,0.032619,0.069500,1
20193,2020-06-09,AXP,110.080002,110.459999,5517400.0,0.012436,0.890337,0.097227,-1


In [ ]:
big_df.to_csv('/content/drive/MyDrive/Dataset/Datasets/sp500.csv')

In [ ]:
dates = big_df.date.unique()

In [ ]:
len(dates)

2609

In [ ]:
big_df.shape

(20195, 9)

In [ ]:
sp500 =  pdr.get_data_yahoo("^GSPC", "2010-01-01", "2021-01-01")

[*********************100%***********************]  1 of 1 completed


In [ ]:
sp500

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,1132.989990,3991400000
2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,1136.520020,2491020000
2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,1137.140015,4972660000
2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,1141.689941,5270680000
2010-01-08,1140.520020,1145.390015,1136.219971,1144.979980,1144.979980,4389590000
...,...,...,...,...,...,...
2020-12-24,3694.030029,3703.820068,3689.320068,3703.060059,3703.060059,1885090000
2020-12-28,3723.030029,3740.510010,3723.030029,3735.360107,3735.360107,3527460000
2020-12-29,3750.010010,3756.120117,3723.310059,3727.040039,3727.040039,3387030000
